## MLP From Scratch

A Ideia é criar uma rede neural MLP com uma e duas hidden layers sem a utilização de nenhuma biblioteca dedicada (ex. sklearn) para as funções core da rede e aplicá-la, sem alteração de arquitetura, sobre dois datasets.
- Datasets utilizados:
    - wine.data1 (classificação, https://archive.ics.uci.edu/ml/datasets/Wine)
    - default_features_1059_tracks.txt2 (regressão / aproximação, https://archive.ics.uci.edu/ml/datasets/Geographical+Original+of+Music)
- Métricas: 
    - Acurácia para problema de classificação
    - RMSE para problema de regressão.

##### Importa bibliotecas utilizadas

In [1]:
import numpy as np
import math
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
import pandas as pd
from zipfile import ZipFile
import warnings
from warnings import simplefilter
simplefilter(action='ignore')

##### Define Classe da rede MLP com seus parâmetros e funções

In [2]:
#MLP

class NeuralNetwork(object):

    #Parâmetros da rede 
    def __init__(self, eta = 0.1, epoch =10000, epsilon=0.01, alfa=0, i=13, j=3, k=3, nlayer=1, job='c'):
        self.eta = eta # taxa de aprendizagem
        self.epoch = epoch # número máximo de épocas
        self.epsilon = epsilon # erro médio admissível
        self.alfa=alfa # fator do termo momentum
        self.i=i # número de neurônios na camada de entrada. Deve ter a dimensionalidade das features
        self.j=j # número de neurônios na camada j
        self.k=k # número de neurônios na camada k. Deve ter 
        self.nlayer= nlayer # número de camadas intermediárias
        self.job = job # 'c' para caso de classificação e 'r' para caso de regressão
        if self.job == 'c': # adaptação para caso geral e correto parse do cálculo de erro
            self.l=nlayer+1 # número de neurônios na camada l
        else:
            self.l=nlayer # número de neurônios na camada l

    #Inicializa os pesos (matriz Wji) e bias (vetor teta_j) da rede. 
    #j,i representam os números de neurônios da camada de saída e entrada    
    def inicialize(self): 

        self.w_ji=np.random.uniform(-1,1, [self.j,self.i+1]) # Inicialização com parâmetros randômicos
        self.w_kj=np.random.uniform(-1,1, [self.k,self.j+1]) # Inicialização com parâmetros randômicos

        if (self.nlayer>1):
          self.w_lk=np.random.uniform(-1,1, [self.l,self.k+1]) # Inicialização com parâmetros randômicos

        return self

    #Função de ativação utilizada - sigmoide
    def sigm(self, x):
        return 1/(1 + np.exp(-x))

    #Derivada da função sigmoide
    def d_sigm(self, x):
        return x * (1.0 - x)

    #Cálculo da saída da camada escondida      
    def feed_net(self, y_i):
        self.y_i=y_i #entrada da camada i
        self.y_j=self.sigm(np.dot(self.w_ji,np.concatenate((y_i,[1])))) #entrada da camada j, termo adicionado devido ao bias
        self.y_k=self.sigm(np.dot(self.w_kj,np.concatenate((self.y_j,[1])))) #entrada da camada K, termo adicionado devido ao bias

        if (self.nlayer>1):
          self.y_l=self.sigm(np.dot(self.w_lk,np.concatenate((self.y_k,[1])))) #entrada da camada L, termo adicionado devido ao bias
        
        return self

    #Cálculo dos gradientes  para backpropagation
    def back_propagation(self, error):
        
        if (self.nlayer>1):
          self.delta_l=error*self.d_sigm(self.y_l) #Gradiente local do erro na camada de saída
          self.delta_k=np.dot(self.w_lk.T,self.delta_l)*self.d_sigm(np.concatenate((self.y_k,[1])))#Gradiente local do erro na camada escondida
          self.delta_k=self.delta_k[0:self.k] # O último elemento representava apenas a relação do erro do bias, mas o bias não se interliga com a camada anterior.
          
        else:
          self.delta_k=error*self.d_sigm(self.y_k) #Gradiente local do erro na camada de saída
        
        self.delta_j=np.dot(self.w_kj.T,self.delta_k)*self.d_sigm(np.concatenate((self.y_j,[1]))) #Gradiente local do erro na camada escondida
        self.delta_j=self.delta_j[0:self.j] # O último elemento representava apenas a relação do erro do bias, mas o bias não se interliga com a camada anterior.  

        return self

    #Fit para atualização dos pesos.
    def fit(self, X, y):
        
        emed=1.01 # Inicializa erro aceitável com valor maior que o range usual para garantir primeira iteração
        count=0 # Inicializa contador
        self.inicialize()
        #Inicia delta pesos com valores zerados
        deltaw_lk=0 
        deltaw_kj=0
        deltaw_ji=0

        
        while (emed > self.epsilon and count < self.epoch):
         
         index = np.arange(X.shape[0])
         np.random.shuffle(index)
         X = X[index]
         y = y[index]

         emed=0 #Zera o erro aceitável para posterior atualização
         
         for n in range (np.size(X, 0)):

           y_i=X[n,:]
           self.feed_net(y_i)      
           # Calcula erros 
           if (self.nlayer>1):
             error=y[n,:]-self.y_l
           else:
             error=y[n,:]-self.y_k
           #Atualiza erro médio e calcula gradientes para backpropagation            
           emed += np.sum(np.square(error))
           self.back_propagation(error) 
            #Atualiza os pesos 
           if (self.nlayer>1):
             self.w_lk=self.w_lk+self.eta*self.delta_l[:, np.newaxis]*np.concatenate((self.y_k,[1])) + self.alfa*deltaw_lk
          
           self.w_kj=self.w_kj+self.eta*self.delta_k[:, np.newaxis]*np.concatenate((self.y_j,[1])) + self.alfa*deltaw_kj
           self.w_ji=self.w_ji+self.eta*self.delta_j[:, np.newaxis]*np.concatenate((self.y_i,[1]))+ self.alfa*deltaw_ji

           if (self.nlayer>1):
             deltaw_lk=self.eta*self.delta_l[:, np.newaxis]*np.concatenate((self.y_k,[1])) + self.alfa*deltaw_lk
           
           deltaw_kj=self.eta*self.delta_k[:, np.newaxis]*np.concatenate((self.y_j,[1])) + self.alfa*deltaw_kj
           deltaw_ji=-self.eta*self.delta_j[:, np.newaxis]*np.concatenate((self.y_i,[1])) + self.alfa*deltaw_ji
           
           self.feed_net(y_i)
          
         emed /= np.size(X, 0)
         self.emed=emed
         count += 1
        # Contador para posterior tabulação
        self.counter = count

        return self

    #Define função para realização de predições
    def predict(self, X):
        ypred=[]
        for n in range (np.size(X, 0)):
          y_i=X[n,:]
          self.feed_net(y_i)

          if (self.nlayer>1):
            ypred.append(self.y_l)
          else: 
            ypred.append(self.y_k)

        return np.array(ypred)

In [3]:
# Lê url da tarefa e retorna dataframe
df_data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',
                      sep=",",
                      names = ['Origin','Alcohol','Malic acid','Ash','Alcalinity of ash','Magnesium','Total phenols', 
                               'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue',
                               'OD280/OD315 of diluted wines', 'Proline'])

In [4]:
#Pré-processamento
#Define features e target
X = df_data.drop('Origin', axis=1)
y = df_data['Origin']
#Aplica one hot encoder sobre target dando dimensionalidade correta das classes com geração de vetores linearmente independentes
enc = OneHotEncoder()
y = y.values.reshape(-1,1)
y_ready = enc.fit_transform(y).toarray()
#Aplica std.Scaler e minmax scalers. Assim eliminamos a variância entre os domínios das features e, com a mesma escala
    #evitamos desbalanceamento no cálculo das fnets, já que evita que a derivada da função sigmoidal tenda a zero, o que implicaria
    #em reduzida atualização dos pesos, fazendo-os estagnar e não proporcionando aprendizado.
scaler = StandardScaler()
minmax = MinMaxScaler()
X = scaler.fit_transform(X)
X_ready = minmax.fit_transform(X)

In [5]:
#Define os valores que cada um dos parâmetros assumirá durante as iterações, que terão todas as combinações destes valores
test_size_iter = [0.2, 0.5]
hidden_layer_iter = [1, 2]
epoch_amount_iter = [50, 500]
learning_rate_iter = [0.2, 0.1]
momentum_term_iter = [0.2, 0.5]


#Listas para posterior tabulação
out_train_size = []
out_hidden_layer = []
out_epoch = []
out_learning_rate = []
out_momentum = []
out_train_acc = []
out_test_acc = []

#Gera a combinação de parâmetros para alimentar as iterações sobre a rede
for test_size in test_size_iter:
    #Divide os arrays de treino, testes, features e targets com random state fixo, para posterior incremento.
    X_train, X_test, y_train, y_test = train_test_split(X_ready, y_ready, test_size=test_size, random_state=8)
    for hidden_layer in hidden_layer_iter:
        for epoch_amount in epoch_amount_iter:
            for learning_rate in learning_rate_iter:
                for momentum_term in momentum_term_iter:
                    train_size = 1-test_size #Calcula proporção de treino para posterior tabulação
                    print ('train_size = ', train_size) #Imprime parâmetros a cada iteração
                    print ('hidden_layers = ', hidden_layer) #Imprime parâmetros a cada iteração
                    print ('available_epochs = ', epoch_amount) #Imprime parâmetros a cada iteração
                    print ('learning_rate = ', learning_rate) #Imprime parâmetros a cada iteração
                    print ('momentum_term = ', momentum_term) #Imprime parâmetros a cada iteração
                    #Gera a rede com os parâmetros acima descritos
                    nn = NeuralNetwork(eta=learning_rate,
                                       epoch=epoch_amount, 
                                       epsilon=0.005, 
                                       alfa=momentum_term, 
                                       i=X.shape[1], j=10, k=3, 
                                       nlayer=hidden_layer, 
                                       job='c')
                    fitted = nn.fit(X_train,y_train) #Atualização dos pesos
                    y_train_pred = np.around(nn.predict(X_train)) #Predição sobre grupo de treino
                    y_test_pred = np.around(nn.predict(X_test)) #Predição sobre grupo de teste
                    train_acc = round(accuracy_score(y_train, y_train_pred),3) #Cálculo da acurácia de treino para posterior tabulação
                    test_acc = round(accuracy_score(y_test, y_test_pred),3) #Cálculo da acuracia de teste para posterior tabulação
                    print ('epochs_used = ', fitted.counter) #Imprime quantidades de ciclos utilizados a cada iteração até que 
                                                                #o valor de erro aceitável seja atingido ou que o número de ciclos
                                                                #disponíveis se esgote
                    print ('accuracy_train_set =', train_acc) #Imprime acurácia de treino a cada iteração
                    print ('accuracy_test_set =', test_acc) #Imprime acurácia de teste a cada iteração
                    #Atualiza listas com valores da iteração para posterior tabulação
                    out_train_size.append(train_size) 
                    out_hidden_layer.append(hidden_layer)
                    out_epoch.append(fitted.counter)
                    out_learning_rate.append(learning_rate)
                    out_momentum.append(momentum_term)
                    out_train_acc.append(train_acc)
                    out_test_acc.append(test_acc)
                    print ('+_+_+_+_+_+_+') #Imprime fim da iteração

train_size =  0.8
hidden_layers =  1
available_epochs =  50
learning_rate =  0.2
momentum_term =  0.2
epochs_used =  50
accuracy_train_set = 0.993
accuracy_test_set = 0.889
+_+_+_+_+_+_+
train_size =  0.8
hidden_layers =  1
available_epochs =  50
learning_rate =  0.2
momentum_term =  0.5
epochs_used =  50
accuracy_train_set = 0.81
accuracy_test_set = 0.778
+_+_+_+_+_+_+
train_size =  0.8
hidden_layers =  1
available_epochs =  50
learning_rate =  0.1
momentum_term =  0.2
epochs_used =  50
accuracy_train_set = 0.979
accuracy_test_set = 0.889
+_+_+_+_+_+_+
train_size =  0.8
hidden_layers =  1
available_epochs =  50
learning_rate =  0.1
momentum_term =  0.5
epochs_used =  50
accuracy_train_set = 0.866
accuracy_test_set = 0.722
+_+_+_+_+_+_+
train_size =  0.8
hidden_layers =  1
available_epochs =  500
learning_rate =  0.2
momentum_term =  0.2
epochs_used =  223
accuracy_train_set = 1.0
accuracy_test_set = 0.972
+_+_+_+_+_+_+
train_size =  0.8
hidden_layers =  1
available_epochs =  500
learn

In [6]:
#Monta dataframe tabulado para posterior relatório
df_table_class = pd.DataFrame()
df_table_class['Percentual Treino'] = out_train_size
df_table_class['Camadas Intermediárias'] = out_hidden_layer
df_table_class['Ciclos para Treinamento'] = out_epoch
df_table_class['Velocidade de Aprendizado'] = out_learning_rate
df_table_class['Termo Momentum'] = out_momentum
df_table_class['Acurácia Dados Treino'] = out_train_acc
df_table_class['Acurácia Dados Teste'] = out_test_acc

In [7]:
#Imprime dataframe com valores de acurácia de dados de teste descendentes
df_table_class = df_table_class.sort_values(by='Acurácia Dados Teste', ascending=False, ignore_index=True)
df_table_class.head(50)

,Percentual Treino,Camadas Intermediárias,Ciclos para Treinamento,Velocidade de Aprendizado,Termo Momentum,Acurácia Dados Treino,Acurácia Dados Teste
0,0.5,2,50,0.2,0.2,0.978,0.978
1,0.5,2,376,0.1,0.2,1.000,0.978
2,0.5,2,171,0.2,0.2,1.000,0.978
3,0.5,1,500,0.1,0.2,1.000,0.978
4,0.8,2,50,0.2,0.2,0.993,0.972
5,0.8,1,223,0.2,0.2,1.000,0.972
6,0.8,1,435,0.1,0.2,1.000,0.972
7,0.5,1,50,0.2,0.2,0.978,0.966
8,0.5,2,50,0.1,0.2,0.989,0.966
9,0.5,1,336,0.2,0.2,1.000,0.966


In [8]:
#Lê a url disponibilizada, baixa o arquivo .zip e o extrai para pasta C:/temp_work
import requests, zipfile, io
r = requests.get('https://archive.ics.uci.edu/ml/machine-learning-databases/00315/Geographical%20Original%20of%20Music.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('C:/temp_work')

In [9]:
#Cria dataframe com o arquivo de interesse conforme orientação da tarefa
df = pd. read_csv (r'C:/temp_work/Geographical Original of Music/default_features_1059_tracks.txt', header=None)

In [10]:
#Pré-processamento
#Define features e target
X = df.drop(df.columns[-2:], axis=1)
y = df.drop(df.columns[:-2], axis=1)
#Aplica std.Scaler e minmax scalers. Assim eliminamos a variância entre os domínios das features e, com a mesma escala
    #evitamos desbalanceamento no cálculo das fnets, já evita que a derivada da função sigmoidal tenda a zero, o que implicaria
    #em reduzida atualização dos pesos, fazendo-os stagnares e não proporcionando aprendizado.
scaler = StandardScaler()
minmax = MinMaxScaler()
X = scaler.fit_transform(X)
X_ready = minmax.fit_transform(X)
#Cria array do target
y_ready = y.values

In [11]:
#Define os valores que cada um dos parâmetros assumirá durante as iterações, que terão todas as combinações destes valores
test_size_iter = [0.2, 0.5]
hidden_layer_iter = [1, 2]
epoch_amount_iter = [50, 500]
learning_rate_iter = [0.2, 1]
momentum_term_iter = [0.1, 0.5]


#Listas para posterior tabulação
out_train_size = []
out_hidden_layer = []
out_epoch = []
out_learning_rate = []
out_momentum = []
out_train_rmse = []
out_test_rmse = []

#Gera a combinação de parâmetros para alimentar as iterações sobre a rede
for test_size in test_size_iter:
    #Divide os arrays de treino, testes, features e targets com random state fixo, para posterior incremento.
    X_train, X_test, y_train, y_test = train_test_split(X_ready, y_ready, test_size=test_size, random_state=8)
    for hidden_layer in hidden_layer_iter:
        for epoch_amount in epoch_amount_iter:
            for learning_rate in learning_rate_iter:
                for momentum_term in momentum_term_iter:
                    train_size = 1-test_size #Calcula proporção de treino para posterior tabulação
                    print ('train_size = ', train_size) #Imprime parâmetros a cada iteração
                    print ('hidden_layers = ', hidden_layer) #Imprime parâmetros a cada iteração
                    print ('epochs = ', epoch_amount) #Imprime parâmetros a cada iteração
                    print ('learning_rate = ', learning_rate) #Imprime parâmetros a cada iteração
                    print ('momentum_term = ', momentum_term) #Imprime parâmetros a cada iteração
                    #Gera a rede com os parâmetros acima descritos
                    nn = NeuralNetwork(eta=learning_rate,
                                       epoch=epoch_amount, 
                                       epsilon=0.005, 
                                       alfa=momentum_term, 
                                       i=X.shape[1], j=38, k=2, 
                                       nlayer=hidden_layer, 
                                       job='r')
                    fitted = nn.fit(X_train,y_train) #Atualização dos pesos
                    y_train_pred = nn.predict(X_train) #Predição sobre grupo de treino
                    y_test_pred = nn.predict(X_test) #Predição sobre grupo de teste
                    train_rmse = round(np.sqrt(mean_squared_error(y_train, y_train_pred)),3) #Cálculo de RMSE de treino para posterior tabulação
                    test_rmse = round(np.sqrt(mean_squared_error(y_test, y_test_pred)),3) #Cálculo de RMSE de treino para posterior tabulação
                    print ('epochs_used = ', fitted.counter) #Imprime quantidades de ciclos utilizados a cada iteração até que 
                                                                #o valor de erro aceitável seja atingido ou que o número de ciclos
                                                                #disponíveis se esgote
                    print ('RMSE_train_set =', train_rmse) #Imprime RMSE a cada iteração
                    print ('RMSE_test_set =', test_rmse) #Imprime RMSE a cada iteração
                    #Atualiza listas com valores da iteração para posterior tabulação
                    out_train_size.append(train_size)
                    out_hidden_layer.append(hidden_layer)
                    out_epoch.append(epoch_amount)
                    out_learning_rate.append(learning_rate)
                    out_momentum.append(momentum_term)
                    out_train_rmse.append(train_rmse)
                    out_test_rmse.append(test_rmse)
                    print ('+_+_+_+_+_+_+') #Imprime fim da iteração

train_size =  0.8
hidden_layers =  1
epochs =  50
learning_rate =  0.2
momentum_term =  0.1
epochs_used =  50
RMSE_train_set = 49.633
RMSE_test_set = 49.895
+_+_+_+_+_+_+
train_size =  0.8
hidden_layers =  1
epochs =  50
learning_rate =  0.2
momentum_term =  0.5
epochs_used =  50
RMSE_train_set = 49.633
RMSE_test_set = 49.895
+_+_+_+_+_+_+
train_size =  0.8
hidden_layers =  1
epochs =  50
learning_rate =  1
momentum_term =  0.1
epochs_used =  50
RMSE_train_set = 50.272
RMSE_test_set = 50.542
+_+_+_+_+_+_+
train_size =  0.8
hidden_layers =  1
epochs =  50
learning_rate =  1
momentum_term =  0.5
epochs_used =  50
RMSE_train_set = 49.633
RMSE_test_set = 49.895
+_+_+_+_+_+_+
train_size =  0.8
hidden_layers =  1
epochs =  500
learning_rate =  0.2
momentum_term =  0.1
epochs_used =  500
RMSE_train_set = 49.633
RMSE_test_set = 49.895
+_+_+_+_+_+_+
train_size =  0.8
hidden_layers =  1
epochs =  500
learning_rate =  0.2
momentum_term =  0.5
epochs_used =  500
RMSE_train_set = 49.633
RMSE_test_s

In [12]:
#Monta dataframe tabulado para posterior relatório
df_table_reg = pd.DataFrame()
df_table_reg['Percentual Treino'] = out_train_size
df_table_reg['Camadas Intermediárias'] = out_hidden_layer
df_table_reg['Ciclos para Treinamento'] = out_epoch
df_table_reg['Velocidade de Aprendizado'] = out_learning_rate
df_table_reg['Termo Momentum'] = out_momentum
df_table_reg['RMSE Dados Treino'] = out_train_rmse
df_table_reg ['RMSE Dados Teste'] = out_test_rmse

In [13]:
#Imprime dataframe com valores de RMSE de dados de teste descendentes
df_table_reg = df_table_reg.sort_values(by='RMSE Dados Teste', ascending=False, ignore_index=True)
df_table_reg.head(50)

,Percentual Treino,Camadas Intermediárias,Ciclos para Treinamento,Velocidade de Aprendizado,Termo Momentum,RMSE Dados Treino,RMSE Dados Teste
0,0.8,1,50,1.0,0.1,50.272,50.542
1,0.8,2,500,1.0,0.5,50.011,50.285
2,0.8,1,500,1.0,0.1,50.011,50.285
3,0.8,2,50,1.0,0.5,49.897,50.154
4,0.8,1,50,0.2,0.1,49.633,49.895
5,0.8,2,50,1.0,0.1,49.633,49.895
6,0.8,1,50,0.2,0.5,49.633,49.895
7,0.8,2,500,1.0,0.1,49.633,49.895
8,0.8,2,500,0.2,0.5,49.633,49.895
9,0.8,2,500,0.2,0.1,49.633,49.895


### FIM DO SCRIPT

### Conclusões

- Analisando os dados da tabela class além do script disponibilizado, pode-se concluir que, para o caso de classificação, o termo momentum de valor 0,2 foi sistematicamente melhor quando comparado ao valor de 0,5. Quando utilizado o valor 0,2, frequentemente o erro máximo aceitável foi atingido e, eventualmente, não sendo necessários todos os ciclos de treinamento disponibilizados à rede. Além disso pode-se notar que as únicas ocorrências de acurácia maior para termo momentum 0,2 em relação ao termo momentum 0,5 ocorreu com velocidade de aprendizado valor 0,2 e apenas uma camada intermediária, indicando um possível caminho para eventual tunning da rede e nos lembrando que não necessariamente uma rede mais complexa será mais eficiente.
- Já analisando os dados da tabela reg além do script disponibilizado, pode-se concluir que, para o caso de regressão multivariada a rede não foi eficiente, apresentando erro quadrado altíssimo em relação à grandeza observada (latitude e longitude). Apesar deste fato, podemos notar que a rede com proporção de dados de treino de 0,5 foi sistematicamente melhor que a proporção 0,8 (i.e. 80% reino e 20% teste) . Este resultado sugere que a rede projetada precisa de mais alternativas para lidar com um problema de regressão. sugere-se, por exemplo, a adoção de ativadores não sigmoidais, como Unidade Linear Retificada (ReLU) e linear.
- Observando ambas ocorrências podemos tirar algumas conclusões sobre o caso geral, como: não há uma rede ótima que funciona para qualquer problema de predição, já que seus parâmetros e arquitetura definem sua melhor aplicabilidade; a etapa de tunning de uma rede é especialmente importante, tendo em vista a sensibilidade da mesma aos seus hiperparâmetros.
- Sugere-se, para eventual evolução da rede criada, a adição de novos ativadores e mais etapas de tunning sejam realizadas, garantindo a otimização dos hiperparâmetros.
